# Preparation

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

sns.set_style('darkgrid')
sns.set_context('notebook')

In [ ]:
def corrfunc(x, y, **kws):
    r, p = sp.stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate(f"\u03C1 = {r:.2f}\np = {p:.2f}", #unicode code for lowercase rho (ρ)
                xy=(.1, .9), xycoords=ax.transAxes)

# Load C14 models

In [ ]:
#import c14_models
import c14
from c14.models import liver as lm

# Read patient data

In [ ]:
data = pd.read_csv('C14data_liver_samples_20211020.csv')
data = data.groupby(['type', 'sample', 'ploidy', 'pathology']).mean().dropna(how='all').reset_index()
data['age'] = data['Dcoll'] - data['Dbirth']
#data = data.query('type == "hepatocyte" and pathology != ["Y","C"]')
# data = data.query('pathology not in ["Y", "C"]')
exp_data = data
len(data)

# Plot data 

In [ ]:
tt = np.linspace(1930, 2020)
Catm = c14.models.base.Catm()
plt.plot(tt, Catm.lin(tt))
plt.plot(exp_data['Dbirth'], exp_data['d14C'], ls = 'None', marker = 's')
plt.ylim(-0.05, 0.12)
plt.show()

# Individual rates

In [ ]:
#take about 5 hours
ind_rates = []
index =[]
for ind,_ in data.iterrows():
    row = data.loc[[ind]]
    m = lm.POP1()
    edata = c14.exp_data(row)
    op = c14.optimize(m,edata,step_size=0.1)
    for i in  [0.9,-1,-3]:
        resa = sp.optimize.minimize(lambda x: op.Nloglike(x, op.model), [i],method='powell')
        resa['x'] = float(resa['x'])
        ind_rates.append(resa) 
    
    index.append(ind)
individual_rates = pd.DataFrame(ind_rates,index=pd.MultiIndex.from_product([index,['a','b','c']]))

In [ ]:
idx = pd.IndexSlice
with pd.HDFStore('ind_rates') as s:
    individual_rates = s['ind'] 
exp_data['individual_rate'] = 10**individual_rates['x'].mean(level=0)

In [ ]:

exp_data.to_excel('individual_rate.xlsx')

In [ ]:
tt = np.linspace(1930, 2020)
plt.plot(tt, Catm.lin(tt))
plt.scatter(exp_data['Dbirth'], exp_data['d14C'], marker='s', c=np.log(exp_data['individual_rate']))
plt.colorbar()
plt.ylim(-0.05, 0.15)
plt.show()

# Plot rates

In [ ]:
ax = sns.catplot(data=exp_data, x='type', y='individual_rate', kind='box')
ax.set_xticklabels(rotation=30)
plt.show()

In [ ]:
ax = sns.catplot(data=exp_data,  y='individual_rate', kind='box')
plt.show()

# Detect outliers based on IQR

In [ ]:
# Computing IQR
Q1 = exp_data['individual_rate'].quantile(0.25)
Q3 = exp_data['individual_rate'].quantile(0.75)
IQR = Q3 - Q1

# Selecting Values between Q1-1.5IQR and Q3+1.5IQR
exp_data['rate_is_not_outlier'] = exp_data.eval('(@Q1 - 1.5 * @IQR) <= individual_rate <= (@Q3 + 1.5 * @IQR)')

In [ ]:
ax = sns.catplot(data=exp_data.query('rate_is_not_outlier'), x='type', y='individual_rate', kind='box')
# sns.swarmplot(data=exp_data, x='type', y='individual_rate_SSE', color='black', size=2)
ax.set_xticklabels(rotation=30)
plt.show()

# Analysing results

I use the values obtained with SSE

In [ ]:
exp_data['cell_age'] = 1. / exp_data['individual_rate']

In [ ]:
exp_data.head()

In [ ]:
# exp_data = exp_data[exp_data['ploidy'] == '4n'].query('pathology in ["N", "T"]')

In [ ]:
# exp_data

In [ ]:
sns.pairplot(exp_data.query('rate_is_not_outlier').query('type == "hepatocyte"'),
            x_vars = ['subject_age'],
            y_vars = ['cell_age'],
             kind='reg',
            hue = 'ploidy')
plt.ylim(0, 10)
plt.show()

In [ ]:
g = sns.pairplot(exp_data.query('rate_is_not_outlier'),
            x_vars = ['subject_age', 'Dbirth', 'Dcoll'],
            y_vars = ['cell_age', 'individual_rate'],
             kind='reg',
            hue = 'type')
g.axes[0][0].set_ylim(0, 50)
g.axes[1][0].set_ylim(-0.1, 0.6)
plt.show()